In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory


# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from random import seed
from random import random
# seed random number generator
seed(1)

import cv2
import os
files = os.listdir("../input/capstone-good-dataset/Dataset")
i = 0
j=0
images = []
while i < 11000:
    if i < 10:
        temp = "000"+str(i)
    elif i >= 10 and i < 100:
        temp = "00"+str(i)
    elif i >= 100 and i<1000:
        temp = "0"+str(i)
    else:
        temp = str(i)
    i = i+1
    if i>6000:
        if j<4000 and (random()<0.9):
            images.append((cv2.resize(cv2.imread("../input/capstone-good-dataset/Dataset/image_" + temp + ".jpg"), dsize=(320, 180), interpolation=cv2.INTER_CUBIC)))
            j += 1
    else:    
        images.append((cv2.resize(cv2.imread("../input/capstone-good-dataset/Dataset/image_" + temp + ".jpg"), dsize=(320, 180), interpolation=cv2.INTER_CUBIC)))

y = []
j = 0
while j < 11000:
    if j <= 1999:
        y.append("in bed")
    elif j >= 2000 and j <=3999:
        y.append("getting out of bed")
    elif j >= 4000 and j <=5999:
        y.append("out of bed")
    elif j >= 6000 and j <=9999:
        y.append("injured")
    j += 1
    
y = np.array(y)
x_data = np.array(images).reshape(10000, 320, 180, 3)

In [ ]:
import tensorflow as tf
from tensorflow import keras

import matplotlib.pyplot as plt
import os
import time

from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras import layers 
from tensorflow.keras import Model

base_model = VGG16(input_shape = (320, 180, 3),include_top = False, weights = 'imagenet')

for layer in base_model.layers:
    layer.trainable = False
    
# Flatten the output layer to 1 dimension
x = layers.Flatten()(base_model.output)

# Add a fully connected layer with 512 hidden units and ReLU activation
x = layers.Dense(512, activation='relu')(x)

# Add a dropout rate of 0.5
x = layers.Dropout(0.5)(x)

x = layers.Dense(4, activation='softmax')(x)

model = tf.keras.models.Model(base_model.input, x)

model.compile(optimizer = tf.keras.optimizers.RMSprop(lr=0.0001), loss = tf.keras.losses.CategoricalCrossentropy(), metrics=[tf.keras.metrics.AUC(), tf.keras.metrics.CategoricalAccuracy()])

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(y)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
onehot_encoder = OneHotEncoder(sparse=False)
onehot_encoded = onehot_encoder.fit_transform(integer_encoded)
print(onehot_encoded)
X_train, X_test, y_train, y_test = train_test_split(x_data, onehot_encoded, test_size=0.2, random_state=42)

In [ ]:
print(np.shape(y_train))


In [ ]:
#ask if shear or rotation range makes any sense
#no vertical flip since it does not makes sense
#rescale used to normalize image between 0 and 1


datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True,
    #width_shift_range=0.05, #shifts image left/right
    #height_shift_range=0.05, #shift image up/down
    #horizontal_flip=True, 
    #brightness_range = (0.8, 1), #changes brightness, can use this to simulate different lighting intensities
    #zoom_range = [0.9,1.0], #zoom in on image, no zoom out applied
    #rescale=1.0/255.0, #normalizes the pixels
    validation_split=0.2)

training_generator = datagen.flow(X_train, y_train, batch_size=64,subset='training',seed=7)
validation_generator = datagen.flow(X_train, y_train, batch_size=64,subset='validation',seed=7)

loss_history = model.fit_generator(training_generator,
                        validation_data = validation_generator,
                        steps_per_epoch = (len(X_train)*0.8) / 64,
                        validation_steps=(len(X_train)*0.2)/64,
                        epochs=20)

In [ ]:
y_pred_test = model.predict(X_test)

In [ ]:
print(y_test)
print(y_pred_test)

In [ ]:
prediction = []
for element in range(len(y_pred_test)):
    if y_pred_test[element][0] > y_pred_test[element][1] and y_pred_test[element][0] > y_pred_test[element][2] and y_pred_test[element][0] > y_pred_test[element][3]:
        prediction.append([1, 0, 0, 0])
    elif y_pred_test[element][1] > y_pred_test[element][0] and y_pred_test[element][1] > y_pred_test[element][2] and y_pred_test[element][1] > y_pred_test[element][3]:
        prediction.append([0, 1, 0, 0])
    elif y_pred_test[element][2] > y_pred_test[element][1] and y_pred_test[element][2] > y_pred_test[element][0] and y_pred_test[element][2] > y_pred_test[element][3]:
        prediction.append([0, 0, 1, 0])
    else:
        prediction.append([0, 0, 0, 1])
print(prediction)

In [ ]:
from sklearn.metrics import recall_score, precision_score, accuracy_score, f1_score
print('Model accuracy score: {0:0.4f}'. format(accuracy_score((y_test), prediction)))
print('Model precision score: {0:0.4f}'. format(precision_score((y_test), prediction, average = "macro")))
print('Model recall score: {0:0.4f}'. format(recall_score((y_test), prediction, average = "macro")))
print('Model F1 score: {0:0.4f}'. format(f1_score((y_test), prediction, average = "macro")))

In [ ]:
prediction_int = []
true_int = []
for element in range(len(prediction)):
    if prediction[element][0] == 1:
        prediction_int.append(0)
    elif prediction[element][1] == 1:
        prediction_int.append(1)
    elif prediction[element][2] == 1:
        prediction_int.append(2)
    else:
        prediction_int.append(3)

for element in range(len(y_test)):
    if y_test[element][0] == 1:
        true_int.append(0)
    elif y_test[element][1] == 1:
        true_int.append(1)
    elif y_test[element][2] == 1:
        true_int.append(2)
    else:
        true_int.append(3)

In [ ]:
from sklearn.metrics import confusion_matrix
confusion_matrix(true_int, prediction_int)

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras import layers 
from tensorflow.keras import Model

from random import seed
from random import random
# seed random number generator
seed(1)
import numpy as np
import cv2
import os
files = os.listdir("../input/capstone-good-dataset/Dataset")
i = 0
j=0
images = []
while i < 11000:
    if i < 10:
        temp = "000"+str(i)
    elif i >= 10 and i < 100:
        temp = "00"+str(i)
    elif i >= 100 and i<1000:
        temp = "0"+str(i)
    else:
        temp = str(i)
    i = i+1
    if i>6000:
        if j<4000 and (random()<0.9):
            images.append((cv2.resize(cv2.imread("../input/capstone-good-dataset/Dataset/image_" + temp + ".jpg"), dsize=(320, 180), interpolation=cv2.INTER_CUBIC)))
            j += 1
    else:    
        images.append((cv2.resize(cv2.imread("../input/capstone-good-dataset/Dataset/image_" + temp + ".jpg"), dsize=(320, 180), interpolation=cv2.INTER_CUBIC)))

y = []
j = 0
while j < 11000:
    if j <= 1999:
        y.append("in bed")
    elif j >= 2000 and j <=3999:
        y.append("getting out of bed")
    elif j >= 4000 and j <=5999:
        y.append("out of bed")
    elif j >= 6000 and j <=9999:
        y.append("injured")
    j += 1
    
y = np.array(y)
x_data = np.array(images).reshape(10000, 320, 180, 3)


from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(y)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
onehot_encoder = OneHotEncoder(sparse=False)
onehot_encoded = onehot_encoder.fit_transform(integer_encoded)
print(onehot_encoded)
X_train, X_test, y_train, y_test = train_test_split(x_data, onehot_encoded, test_size=0.2, random_state=42)



In [ ]:
base_model = InceptionV3(input_shape = (320, 180, 3),include_top = False, weights = 'imagenet')

for layer in base_model.layers:
    layer.trainable = False
    
# Flatten the output layer to 1 dimension
x = layers.Flatten()(base_model.output)

# Add a fully connected layer with 512 hidden units and ReLU activation
x = layers.Dense(1024, activation='relu')(x)

# Add a dropout rate of 0.5
x = layers.Dropout(0.5)(x)

x = layers.Dense(4, activation='softmax')(x)

model = tf.keras.models.Model(base_model.input, x)

model.compile(optimizer = tf.keras.optimizers.RMSprop(lr=0.000005), loss = tf.keras.losses.CategoricalCrossentropy(), metrics=[tf.keras.metrics.AUC(), tf.keras.metrics.CategoricalAccuracy()])

datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True,
    #width_shift_range=0.05, #shifts image left/right
    #height_shift_range=0.05, #shift image up/down
    #horizontal_flip=True, 
    #brightness_range = (0.8, 1), #changes brightness, can use this to simulate different lighting intensities
    #zoom_range = [0.9,1.0], #zoom in on image, no zoom out applied
    #rescale=1.0/255.0, #normalizes the pixels
    validation_split=0.2)

training_generator = datagen.flow(X_train, y_train, batch_size=64,subset='training',seed=7)
validation_generator = datagen.flow(X_train, y_train, batch_size=64,subset='validation',seed=7)

loss_history = model.fit_generator(training_generator,
                        validation_data = validation_generator,
                        steps_per_epoch = (len(X_train)*0.8) / 64,
                        validation_steps=(len(X_train)*0.2)/64,
                        epochs=150)

y_pred_test = model.predict(X_test)
prediction = []
for element in range(len(y_pred_test)):
    if y_pred_test[element][0] > y_pred_test[element][1] and y_pred_test[element][0] > y_pred_test[element][2] and y_pred_test[element][0] > y_pred_test[element][3]:
        prediction.append([1, 0, 0, 0])
    elif y_pred_test[element][1] > y_pred_test[element][0] and y_pred_test[element][1] > y_pred_test[element][2] and y_pred_test[element][1] > y_pred_test[element][3]:
        prediction.append([0, 1, 0, 0])
    elif y_pred_test[element][2] > y_pred_test[element][1] and y_pred_test[element][2] > y_pred_test[element][0] and y_pred_test[element][2] > y_pred_test[element][3]:
        prediction.append([0, 0, 1, 0])
    else:
        prediction.append([0, 0, 0, 1])
print(prediction)

from sklearn.metrics import recall_score, precision_score, accuracy_score, f1_score
print('Model accuracy score: {0:0.4f}'. format(accuracy_score((y_test), prediction)))
print('Model precision score: {0:0.4f}'. format(precision_score((y_test), prediction, average = "macro")))
print('Model recall score: {0:0.4f}'. format(recall_score((y_test), prediction, average = "macro")))
print('Model F1 score: {0:0.4f}'. format(f1_score((y_test), prediction, average = "macro")))

prediction_int = []
true_int = []
for element in range(len(prediction)):
    if prediction[element][0] == 1:
        prediction_int.append(0)
    elif prediction[element][1] == 1:
        prediction_int.append(1)
    elif prediction[element][2] == 1:
        prediction_int.append(2)
    else:
        prediction_int.append(3)

for element in range(len(y_test)):
    if y_test[element][0] == 1:
        true_int.append(0)
    elif y_test[element][1] == 1:
        true_int.append(1)
    elif y_test[element][2] == 1:
        true_int.append(2)
    else:
        true_int.append(3)
        
from sklearn.metrics import confusion_matrix
confusion_matrix(true_int, prediction_int)

In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers 
from tensorflow.keras import Model

from random import seed
from random import random
# seed random number generator
seed(1)
import numpy as np
import cv2
import os
files = os.listdir("../input/capstone-good-dataset/Dataset")
i = 0
j=0
images = []
while i < 11000:
    if i < 10:
        temp = "000"+str(i)
    elif i >= 10 and i < 100:
        temp = "00"+str(i)
    elif i >= 100 and i<1000:
        temp = "0"+str(i)
    else:
        temp = str(i)
    i = i+1
    if i>6000:
        if j<4000 and (random()<0.9):
            images.append((cv2.resize(cv2.imread("../input/capstone-good-dataset/Dataset/image_" + temp + ".jpg"), dsize=(320, 180), interpolation=cv2.INTER_CUBIC)))
            j += 1
    else:    
        images.append((cv2.resize(cv2.imread("../input/capstone-good-dataset/Dataset/image_" + temp + ".jpg"), dsize=(320, 180), interpolation=cv2.INTER_CUBIC)))

y = []
j = 0
while j < 11000:
    if j <= 1999:
        y.append("in bed")
    elif j >= 2000 and j <=3999:
        y.append("getting out of bed")
    elif j >= 4000 and j <=5999:
        y.append("out of bed")
    elif j >= 6000 and j <=9999:
        y.append("injured")
    j += 1
    
y = np.array(y)
x_data = np.array(images).reshape(10000, 320, 180, 3)


from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(y)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
onehot_encoder = OneHotEncoder(sparse=False)
onehot_encoded = onehot_encoder.fit_transform(integer_encoded)
X_train, X_test, y_train, y_test = train_test_split(x_data, onehot_encoded, test_size=0.2, random_state=42)

In [2]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras import layers 
from tensorflow.keras import Model



base_model = ResNet50(input_shape = (320, 180, 3),include_top = False, weights = 'imagenet')

for layer in base_model.layers:
    layer.trainable = False
    
# Flatten the output layer to 1 dimension
x = layers.Flatten()(base_model.output)

# Add a fully connected layer with 512 hidden units and ReLU activation
x = layers.Dense(1024, activation='relu')(x)

# Add a dropout rate of 0.5
x = layers.Dropout(0.2)(x)

x = layers.Dense(4, activation='softmax')(x)

model = tf.keras.models.Model(base_model.input, x)

model.compile(optimizer = tf.keras.optimizers.RMSprop(lr=0.0002), loss = tf.keras.losses.CategoricalCrossentropy(), metrics=[tf.keras.metrics.AUC(), tf.keras.metrics.CategoricalAccuracy()])

datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True,
    #width_shift_range=0.05, #shifts image left/right
    #height_shift_range=0.05, #shift image up/down
    #horizontal_flip=True, 
    #brightness_range = (0.8, 1), #changes brightness, can use this to simulate different lighting intensities
    #zoom_range = [0.9,1.0], #zoom in on image, no zoom out applied
    #rescale=1.0/255.0, #normalizes the pixels
    validation_split=0.2)

training_generator = datagen.flow(X_train, y_train, batch_size=64,subset='training',seed=7)
validation_generator = datagen.flow(X_train, y_train, batch_size=64,subset='validation',seed=7)

loss_history = model.fit_generator(training_generator,
                        validation_data = validation_generator,
                        steps_per_epoch = (len(X_train)*0.8) / 64,
                        validation_steps=(len(X_train)*0.2)/64,
                        epochs=35)

y_pred_test = model.predict(X_test)
prediction = []
for element in range(len(y_pred_test)):
    if y_pred_test[element][0] > y_pred_test[element][1] and y_pred_test[element][0] > y_pred_test[element][2] and y_pred_test[element][0] > y_pred_test[element][3]:
        prediction.append([1, 0, 0, 0])
    elif y_pred_test[element][1] > y_pred_test[element][0] and y_pred_test[element][1] > y_pred_test[element][2] and y_pred_test[element][1] > y_pred_test[element][3]:
        prediction.append([0, 1, 0, 0])
    elif y_pred_test[element][2] > y_pred_test[element][1] and y_pred_test[element][2] > y_pred_test[element][0] and y_pred_test[element][2] > y_pred_test[element][3]:
        prediction.append([0, 0, 1, 0])
    else:
        prediction.append([0, 0, 0, 1])

from sklearn.metrics import recall_score, precision_score, accuracy_score, f1_score
print('Model accuracy score: {0:0.4f}'. format(accuracy_score((y_test), prediction)))
print('Model precision score: {0:0.4f}'. format(precision_score((y_test), prediction, average = "macro")))
print('Model recall score: {0:0.4f}'. format(recall_score((y_test), prediction, average = "macro")))
print('Model F1 score: {0:0.4f}'. format(f1_score((y_test), prediction, average = "macro")))

prediction_int = []
true_int = []
for element in range(len(prediction)):
    if prediction[element][0] == 1:
        prediction_int.append(0)
    elif prediction[element][1] == 1:
        prediction_int.append(1)
    elif prediction[element][2] == 1:
        prediction_int.append(2)
    else:
        prediction_int.append(3)

for element in range(len(y_test)):
    if y_test[element][0] == 1:
        true_int.append(0)
    elif y_test[element][1] == 1:
        true_int.append(1)
    elif y_test[element][2] == 1:
        true_int.append(2)
    else:
        true_int.append(3)
        
from sklearn.metrics import confusion_matrix
confusion_matrix(true_int, prediction_int)


2022-03-10 18:06:39.701084: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-03-10 18:06:39.830340: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-03-10 18:06:39.831131: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-03-10 18:06:39.832852: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

94781440/94765736 [==============================] - 1s 0us/step


/opt/conda/lib/python3.7/site-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")
/opt/conda/lib/python3.7/site-packages/keras/engine/training.py:1972: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '
/opt/conda/lib/python3.7/site-packages/keras_preprocessing/image/image_data_generator.py:720: UserWarning: This ImageDataGenerator specifies `featurewise_center`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '
/opt/conda/lib/python3.7/site-packages/keras_preprocessing/image/image_data_generator.py:728: UserWarning: This ImageDataGenerator specifies `featurewise_std_normalization`, but it hasn't been fit on any 

Epoch 1/35


2022-03-10 18:06:52.132881: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8005


100/100 [==============================] - 32s 214ms/step - loss: 10.9638 - auc: 0.6804 - categorical_accuracy: 0.4636 - val_loss: 7.6904 - val_auc: 0.6954 - val_categorical_accuracy: 0.4706
Epoch 2/35
100/100 [==============================] - 20s 203ms/step - loss: 1.8878 - auc: 0.8392 - categorical_accuracy: 0.6406 - val_loss: 0.4984 - val_auc: 0.9558 - val_categorical_accuracy: 0.7881
Epoch 3/35
100/100 [==============================] - 20s 203ms/step - loss: 0.8349 - auc: 0.9099 - categorical_accuracy: 0.7250 - val_loss: 0.8916 - val_auc: 0.9099 - val_categorical_accuracy: 0.7056
Epoch 4/35
100/100 [==============================] - 20s 200ms/step - loss: 0.6347 - auc: 0.9390 - categorical_accuracy: 0.7747 - val_loss: 0.3045 - val_auc: 0.9837 - val_categorical_accuracy: 0.8906
Epoch 5/35
100/100 [==============================] - 20s 200ms/step - loss: 0.4885 - auc: 0.9604 - categorical_accuracy: 0.8253 - val_loss: 0.4524 - val_auc: 0.9656 - val_categorical_accuracy: 0.7962
Epoch

array([[378,  14,   0,   1],
       [ 32, 373,   5,   0],
       [  0,   0, 793,   9],
       [  6,   0,  27, 362]])

In [19]:
loss_history = model.fit_generator(training_generator,
                        validation_data = validation_generator,
                        steps_per_epoch = (len(X_train)*0.8) / 64,
                        validation_steps=(len(X_train)*0.2)/64,
                        epochs=1)

y_pred_test = model.predict(X_test)
prediction = []
for element in range(len(y_pred_test)):
    if y_pred_test[element][0] > y_pred_test[element][1] and y_pred_test[element][0] > y_pred_test[element][2] and y_pred_test[element][0] > y_pred_test[element][3]:
        prediction.append([1, 0, 0, 0])
    elif y_pred_test[element][1] > y_pred_test[element][0] and y_pred_test[element][1] > y_pred_test[element][2] and y_pred_test[element][1] > y_pred_test[element][3]:
        prediction.append([0, 1, 0, 0])
    elif y_pred_test[element][2] > y_pred_test[element][1] and y_pred_test[element][2] > y_pred_test[element][0] and y_pred_test[element][2] > y_pred_test[element][3]:
        prediction.append([0, 0, 1, 0])
    else:
        prediction.append([0, 0, 0, 1])

from sklearn.metrics import recall_score, precision_score, accuracy_score, f1_score
print('Model accuracy score: {0:0.4f}'. format(accuracy_score((y_test), prediction)))
print('Model precision score: {0:0.4f}'. format(precision_score((y_test), prediction, average = "macro")))
print('Model recall score: {0:0.4f}'. format(recall_score((y_test), prediction, average = "macro")))
print('Model F1 score: {0:0.4f}'. format(f1_score((y_test), prediction, average = "macro")))

prediction_int = []
true_int = []
for element in range(len(prediction)):
    if prediction[element][0] == 1:
        prediction_int.append(0)
    elif prediction[element][1] == 1:
        prediction_int.append(1)
    elif prediction[element][2] == 1:
        prediction_int.append(2)
    else:
        prediction_int.append(3)

for element in range(len(y_test)):
    if y_test[element][0] == 1:
        true_int.append(0)
    elif y_test[element][1] == 1:
        true_int.append(1)
    elif y_test[element][2] == 1:
        true_int.append(2)
    else:
        true_int.append(3)
        
from sklearn.metrics import confusion_matrix
confusion_matrix(true_int, prediction_int)


100/100 [==============================] - 20s 201ms/step - loss: 0.0294 - auc: 0.9990 - categorical_accuracy: 0.9923 - val_loss: 0.1654 - val_auc: 0.9930 - val_categorical_accuracy: 0.9569
Model accuracy score: 0.9610
Model precision score: 0.9589
Model recall score: 0.9535
Model F1 score: 0.9559


array([[371,  22,   0,   0],
       [ 14, 394,   2,   0],
       [  0,   0, 794,   8],
       [  9,   1,  22, 363]])

In [16]:
import gc
#del integer_encoded,onehot_encoded
del images
gc.collect()

790

In [20]:
import gc
del X_train
gc.collect()

tf.keras.models.save_model(
    model, 'model_capstone', overwrite=True, include_optimizer=True, save_format=None,
    signatures=None, options=None, save_traces=True
)

2022-03-10 18:32:30.809256: W tensorflow/python/util/util.cc:348] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.
2022-03-10 18:32:44.243501: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 503316480 exceeds 10% of free system memory.
2022-03-10 18:32:44.358824: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 503316480 exceeds 10% of free system memory.
/opt/conda/lib/python3.7/site-packages/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


In [22]:
print("done")
import shutil
shutil.make_archive(capstone_model, 'zip', "./model_capstone")

done


NameError: name 'capstone_model' is not defined

<a href="./model_capstone"> Download File </a>

In [ ]:
from tensorflow.keras.utils import plot_model

plot_model(model, to_file='model_plot.png', show_shapes=True, show_layer_names=True)

2022-03-10 17:16:48.825877: W tensorflow/python/util/util.cc:348] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras import layers 
from tensorflow.keras import Model

from random import seed
from random import random
# seed random number generator
seed(1)
import numpy as np
import cv2
import os
files = os.listdir("../input/capstone-good-dataset/Dataset")
i = 0
j=0
images = []
while i < 11000:
    if i < 10:
        temp = "000"+str(i)
    elif i >= 10 and i < 100:
        temp = "00"+str(i)
    elif i >= 100 and i<1000:
        temp = "0"+str(i)
    else:
        temp = str(i)
    i = i+1
    if i>6000:
        if j<4000 and (random()<0.9):
            images.append((cv2.resize(cv2.imread("../input/capstone-good-dataset/Dataset/image_" + temp + ".jpg"), dsize=(224, 224), interpolation=cv2.INTER_CUBIC)))
            j += 1
    else:    
        images.append((cv2.resize(cv2.imread("../input/capstone-good-dataset/Dataset/image_" + temp + ".jpg"), dsize=(224, 224), interpolation=cv2.INTER_CUBIC)))

y = []
j = 0
while j < 11000:
    if j <= 1999:
        y.append("in bed")
    elif j >= 2000 and j <=3999:
        y.append("getting out of bed")
    elif j >= 4000 and j <=5999:
        y.append("out of bed")
    elif j >= 6000 and j <=9999:
        y.append("injured")
    j += 1
    
y = np.array(y)
x_data = np.array(images).reshape(10000, 224, 224, 3)


from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(y)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
onehot_encoder = OneHotEncoder(sparse=False)
onehot_encoded = onehot_encoder.fit_transform(integer_encoded)
print(onehot_encoded)
X_train, X_test, y_train, y_test = train_test_split(x_data, onehot_encoded, test_size=0.2, random_state=42)






In [ ]:
from tensorflow.keras import layers 
from tensorflow.keras import Model


from tensorflow.keras.applications import EfficientNetB0
base_model = EfficientNetB0(weights='imagenet')

for layer in base_model.layers:
    layer.trainable = False
    
# Flatten the output layer to 1 dimension
x = layers.Flatten()(base_model.output)

# Add a fully connected layer with 512 hidden units and ReLU activation
x = layers.Dense(1024, activation='relu')(x)

# Add a dropout rate of 0.5
x = layers.Dropout(0.2)(x)

x = layers.Dense(4, activation='softmax')(x)

model = tf.keras.models.Model(base_model.input, x)

model.compile(optimizer = tf.keras.optimizers.RMSprop(lr=0.02), loss = tf.keras.losses.CategoricalCrossentropy(), metrics=[tf.keras.metrics.AUC(), tf.keras.metrics.CategoricalAccuracy()])

datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True,
    #width_shift_range=0.05, #shifts image left/right
    #height_shift_range=0.05, #shift image up/down
    #horizontal_flip=True, 
    #brightness_range = (0.8, 1), #changes brightness, can use this to simulate different lighting intensities
    #zoom_range = [0.9,1.0], #zoom in on image, no zoom out applied
    #rescale=1.0/255.0, #normalizes the pixels
    validation_split=0.2)

training_generator = datagen.flow(X_train, y_train, batch_size=64,subset='training',seed=7)
validation_generator = datagen.flow(X_train, y_train, batch_size=64,subset='validation',seed=7)

loss_history = model.fit_generator(training_generator,
                        validation_data = validation_generator,
                        steps_per_epoch = (len(X_train)*0.8) / 64,
                        validation_steps=(len(X_train)*0.2)/64,
                        epochs=150)

y_pred_test = model.predict(X_test)
prediction = []
for element in range(len(y_pred_test)):
    if y_pred_test[element][0] > y_pred_test[element][1] and y_pred_test[element][0] > y_pred_test[element][2] and y_pred_test[element][0] > y_pred_test[element][3]:
        prediction.append([1, 0, 0, 0])
    elif y_pred_test[element][1] > y_pred_test[element][0] and y_pred_test[element][1] > y_pred_test[element][2] and y_pred_test[element][1] > y_pred_test[element][3]:
        prediction.append([0, 1, 0, 0])
    elif y_pred_test[element][2] > y_pred_test[element][1] and y_pred_test[element][2] > y_pred_test[element][0] and y_pred_test[element][2] > y_pred_test[element][3]:
        prediction.append([0, 0, 1, 0])
    else:
        prediction.append([0, 0, 0, 1])
print(prediction)

from sklearn.metrics import recall_score, precision_score, accuracy_score, f1_score
print('Model accuracy score: {0:0.4f}'. format(accuracy_score((y_test), prediction)))
print('Model precision score: {0:0.4f}'. format(precision_score((y_test), prediction, average = "macro")))
print('Model recall score: {0:0.4f}'. format(recall_score((y_test), prediction, average = "macro")))
print('Model F1 score: {0:0.4f}'. format(f1_score((y_test), prediction, average = "macro")))

prediction_int = []
true_int = []
for element in range(len(prediction)):
    if prediction[element][0] == 1:
        prediction_int.append(0)
    elif prediction[element][1] == 1:
        prediction_int.append(1)
    elif prediction[element][2] == 1:
        prediction_int.append(2)
    else:
        prediction_int.append(3)

for element in range(len(y_test)):
    if y_test[element][0] == 1:
        true_int.append(0)
    elif y_test[element][1] == 1:
        true_int.append(1)
    elif y_test[element][2] == 1:
        true_int.append(2)
    else:
        true_int.append(3)
        
from sklearn.metrics import confusion_matrix
confusion_matrix(true_int, prediction_int)